# WEB SCRAPING-ASSIGNMENT3

Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars.

In [1]:
#importing necessary libraries
import selenium
import pandas as pd
from bs4 import BeautifulSoup
import time

#Importing requests
import requests

# importing regex
import re

# Importing selenium webdriver
from selenium import webdriver

# Importing required Exceptions
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait


In [2]:
#Activating the chrome browser
driver = webdriver.Chrome(r"C:/Users/hp/Downloads/chromedriver.exe")

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/1386853290.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:/Users/hp/Downloads/chromedriver.exe")


In [3]:
#Getting amazon page
driver.get('https://www.amazon.in/')
#Giving input
search_item= input('Enter the product that has to be searched : ')

Enter the product that has to be searched : Laptop


In [4]:
#Scraping search bar Xpath and clicking on search icon
search_bar = driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')   
search_bar.clear()                                               
search_bar.send_keys(search_item)
search_button = driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
search_button.click() 

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/438075249.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/438075249.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_button = driver.find_element_by_xpath('/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')


In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“

In [39]:
start_page = 0
end_page = 3
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element_by_xpath('//li[@class="a-disabled a-last"]/a').text == 'Next→':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver.get(next_page)                               # ReLoading the next page

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/3525066509.py:6: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  page_urls = driver.find_elements_by_xpath('//a[@class="a-link-normal s-no-outline"]')


Product urls of page 1 has been scraped.


C:\Users\hp\AppData\Local\Temp/ipykernel_1180/3525066509.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  nxt_button = driver.find_element_by_xpath('//li[@class="a-last"]/a')      # Locating the next_button which is active


Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [51]:
prod_dict = {}
prod_dict['Brand']=[]
prod_dict['Price']=[]
prod_dict['Return/Exchange']=[]
prod_dict['Expected Delivery']=[] 
prod_dict['Availability']=[]
prod_dict['Other Details']=[]
prod_dict['URL']=[]

In [60]:
for url in urls[:30]:
    driver.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    
    
    try:
        brand = driver.find_element_by_xpath('//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')

    
    try:
        price = driver.find_element_by_xpath('//span[@class="a-offscreen"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        prod_dict['Return/Exchange'].append(ret.text)
    except NoSuchElementException:
        prod_dict['Return/Exchange'].append('-')
    try:
        delivry = driver.find_element_by_xpath('//div[@id="ddmDeliveryMessage"]/b')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver.find_element_by_xpath('//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)    

Scraping URL =  https://www.amazon.in/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_next_aps_sr_pg12_1?ie=UTF8&adId=A09857152SNV0B5L1W7JU&url=%2FRenewed-HP-ZBook-Mobile-Workstation%2Fdp%2FB09124V3JR%2Fref%3Dsr_1_177_sspa%3Fcrid%3D2QTREN9P60N94%26keywords%3DLaptop%26qid%3D1640787174%26sprefix%3D%252Caps%252C178%26sr%3D8-177-spons%26psc%3D1&qualifier=1640787174&id=1939887063857400&widgetName=sp_atf_next


C:\Users\hp\AppData\Local\Temp/ipykernel_1180/2164397339.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  brand = driver.find_element_by_xpath('//h1[@id="title"]/span')      # Extracting Name from xpath
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/2164397339.py:16: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  price = driver.find_element_by_xpath('//span[@class="a-offscreen"]')            # Extracting Price from xpath
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/2164397339.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ret = driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/2164397339.py:26: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  delivry = driver.find_element_by_xpath('//div[@id="ddmD

Scraping URL =  https://www.amazon.in/ASUS-VivoBook-i3-1115G4-Integrated-TP470EA-EC301TS/dp/B08CDV49P2/ref=sr_1_178?crid=2QTREN9P60N94&keywords=Laptop&qid=1640787174&sprefix=%2Caps%2C178&sr=8-178
Scraping URL =  https://www.amazon.in/HP-Laptop-14-Inch-Windows-dy2501tu/dp/B093VZC5X9/ref=sr_1_179?crid=2QTREN9P60N94&keywords=Laptop&qid=1640787174&sprefix=%2Caps%2C178&sr=8-179
Scraping URL =  https://www.amazon.in/Acer-A515-56-Display-i5-1135G7-Processor/dp/B096SN1LYF/ref=sr_1_180?crid=2QTREN9P60N94&keywords=Laptop&qid=1640787174&sprefix=%2Caps%2C178&sr=8-180
Scraping URL =  https://www.amazon.in/Acer-Aspire-Processor-inches-Business/dp/B091HGK1B6/ref=sr_1_181?crid=2QTREN9P60N94&keywords=Laptop&qid=1640787174&sprefix=%2Caps%2C178&sr=8-181
Scraping URL =  https://www.amazon.in/HP-3-3250-Laptop-Windows-15s-gr0011AU/dp/B08T6SJ215/ref=sr_1_177?crid=2QTREN9P60N94&keywords=Laptop&qid=1640787174&sprefix=%2Caps%2C178&sr=8-177
Scraping URL =  https://www.amazon.in/HP-53L46PA-Notebook-I3-1005G1-Warr

In [61]:
prod_df = pd.DataFrame.from_dict(prod_dict)
prod_df


,Brand,Price,Return/Exchange,Expected Delivery,Availability,Other Details,URL
0,(Renewed) HP ZBook 15 G3 Mobile Workstation - ...,-,7 Days Replacement,"Friday, Dec 31",In stock.,zbookg3-64GB-2TB || Operating System: Window...,https://www.amazon.in/gp/slredirect/picassoRed...
1,ASUS VivoBook Flip 14 (2020) Intel Core i3-111...,-,7 Days Replacement,"Monday, Jan 3",Only 2 left in stock.,Intel Core i3-1115G4 Processor 3.0 GHz (6M Cac...,https://www.amazon.in/ASUS-VivoBook-i3-1115G4-...
2,HP 14 (2021) 11th Gen Intel Core i3 Laptop wit...,-,7 Days Replacement,-,,Free upgrade to Windows 11 when available* Dis...,https://www.amazon.in/HP-Laptop-14-Inch-Window...
3,Acer Aspire 5 A515-56 Thin and Light Laptop | ...,-,7 Days Replacement,"Monday, Jan 3",Only 2 left in stock.,𝗣𝗼𝘄𝗲𝗿𝗳𝘂𝗹 𝗣𝗿𝗼𝗱𝘂𝗰𝘁𝗶𝘃𝗶𝘁𝘆 𝗮𝗻𝗱 𝗔𝗺𝗮𝘇𝗶𝗻𝗴 𝗕𝘂𝗶𝗹𝗱 𝗤𝘂𝗮𝗹𝗶𝘁...,https://www.amazon.in/Acer-A515-56-Display-i5-...
4,(Renewed) HP ZBook 15 G3 Mobile Workstation - ...,-,7 Days Replacement,"Friday, Dec 31",In stock.,zbookg3-64GB-2TB || Operating System: Window...,https://www.amazon.in/gp/slredirect/picassoRed...
...,...,...,...,...,...,...,...
93,Acer Aspire 3 AMD Ryzen 3 3250U Processor 15.6...,,7 Days Replacement,"Saturday, Jan 1",In stock.,Processor : AMD Ryzen 3 3250U dual-core proces...,https://www.amazon.in/Acer-Processor-Business-...
94,HP Notebook PC 245 G7 14 inches Laptop AMD ATH...,,7 Days Replacement,-,,"HP ATHLON-3050U/4GB/1TB/Windows 10 Home/14"" |...",https://www.amazon.in/HP-Notebook-245-G7-ATHLO...
95,Lenovo IdeaPad Gaming 3 10th Gen Intel i5 15.6...,,7 Days Replacement,"Friday, Dec 31",In stock.,Processor: 10th Gen Intel Core i5 10300H | Spe...,https://www.amazon.in/Lenovo-IdeaPad-Graphics-...
96,Microsoft Surface Laptop 4 AMD Ryzen™ 5 4680U ...,,7 Days Replacement,"Friday, Dec 31",In stock.,Free upgrade to Windows 11 when available. Dis...,https://www.amazon.in/gp/slredirect/picassoRed...


In [64]:

prod_df.to_csv('Amazon_{}.csv'.format(search_item))


3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’

# searching for 10 fruits images

In [147]:
# Opening images.google.com
driver.get('https://images.google.com/')

In [148]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("fruits")       # Inputing "fruits" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button


C:\Users\hp\AppData\Local\Temp/ipykernel_1180/938143929.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/938143929.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button


In [162]:
print("start scrolling to generate more images on the page.....")
# 50 time we scroll down by 10000 in order to generate more images on the website
for _ in range(50):
    driver.execute_script("window.scrollBy(0,10000)")

start scrolling to generate more images on the page.....


In [166]:
 images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')


C:\Users\hp\AppData\Local\Temp/ipykernel_1180/24907920.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')


In [167]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

75

In [168]:
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
    file = open("E:/Images flip robo/fruits/img"+str(i)+".jpg", "wb")
    file.write(response.content)

# Searching for 100 car images

In [169]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("cars")       # Inputing "cars" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button


C:\Users\hp\AppData\Local\Temp/ipykernel_1180/3994406668.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/3994406668.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button


In [170]:
print("start scrolling to generate more images on the page.....")
# 50 time we scroll down by 10000 in order to generate more images on the website
for _ in range(50):
    driver.execute_script("window.scrollBy(0,10000)")

start scrolling to generate more images on the page.....


In [172]:
 images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/206814363.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')


In [173]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

68

In [174]:
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
    file = open("E:/Images flip robo/cars/img"+str(i)+".jpg", "wb")
    file.write(response.content)

# searching for 100  ‘Machine Learning’ images

In [175]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("Machine Learning")       # Inputing "Machine Learning" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/1686637617.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/1686637617.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button


In [176]:
print("start scrolling to generate more images on the page.....")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

start scrolling to generate more images on the page.....


In [178]:
 images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/206814363.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')


In [179]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

83

In [181]:
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
    file = open("E:/Images flip robo/Machine learning/img"+str(i)+".jpg", "wb")
    file.write(response.content)

# searching for 100 ‘Guitar’ images

In [182]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("Guitar")       # Inputing "Guitar" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/3106995442.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/3106995442.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button


In [191]:
print("start scrolling to generate more images on the page.....")
# 500 time we scroll down by 10000 in order to generate more images on the website
for _ in range(500):
    driver.execute_script("window.scrollBy(0,10000)")

start scrolling to generate more images on the page.....


In [192]:
 images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/206814363.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')


In [193]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

31

In [194]:
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
    file = open("E:/Images flip robo/Guitar/img"+str(i)+".jpg", "wb")
    file.write(response.content)

# searching for 10 ‘Cakes’ images

In [235]:
search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
search_bar.send_keys("Cakes")       # Inputing "Cakes" keyword to search rock images
search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/496596930.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')    # Finding the search bar using it's xpath
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/496596930.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_button = driver.find_element_by_xpath('//*[@id="sbtc"]/button')    # Finding the xpath of search button


In [239]:
print("start scrolling to generate more images on the page.....")
# 50 time we scroll down by 1000 in order to generate more images on the website
for _ in range(50):
    driver.execute_script("window.scrollBy(0,1000)")

start scrolling to generate more images on the page.....


In [243]:
 images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/206814363.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  images = driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]')


In [244]:
img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

35

In [245]:
for i in range(len(img_urls)):
    if i >= 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response= requests.get(img_urls[i])
    file = open("E:/Images flip robo/cakes/img"+str(i)+".jpg", "wb")
    file.write(response.content)

Q-4 Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [251]:
#Display Resolution”, “Processor”, “Processor Cores
# Asking for user input
prod = input(" Enter the name of the mobile phone you want to search : ")
driver.get('https://www.flipkart.com/')
time.sleep(3)
try:
    login_X_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')                      # Button to close login popup
    login_X_button.click()
except NoSuchElementException : 
    print("No Login page")
search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')    # Finding the search bar using it's xpath
search_bar.clear()               # Clearing the search bar
search_bar.send_keys(prod)       # Inputing keyword to search
search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

 Enter the name of the mobile phone you want to search : pixel 4A


C:\Users\hp\AppData\Local\Temp/ipykernel_1180/4097866137.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  login_X_button = driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]')                      # Button to close login popup
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/4097866137.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_xpath('//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')    # Finding the search bar using it's xpath


No Login page


C:\Users\hp\AppData\Local\Temp/ipykernel_1180/4097866137.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_button = driver.find_element_by_xpath('//button[@class="L0Z3Pu"]')    # Finding the xpath of search button


In [253]:
# Fetching urls of phones coming on 1st page
flip_urls = []
urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')
for url in urls:
    flip_urls.append(url.get_attribute("href"))
len(flip_urls)    

C:\Users\hp\AppData\Local\Temp/ipykernel_1180/517229814.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  urls = driver.find_elements_by_xpath('//a[@class="_1fQZEK"]')


24

In [254]:
flip_dict = {}
flip_dict["Brand"] = []
flip_dict["Smartphone"] = []
flip_dict["Colour"] = []
flip_dict["RAM"] = []
flip_dict["Storage(ROM)"] = []
flip_dict["Primary Camera"] = []
flip_dict["Secondary Camera"] = []
flip_dict["Display Size"] = []
flip_dict["Battery Capacity"] = []
flip_dict["Battery Type"] = []
flip_dict["Price"] = []
flip_dict["URL"] = []

In [255]:
# Scraping data from each url
for url in flip_urls:
    driver.get(url)    # Saving url                                                     
    print("Scraping URL = ", url)
    flip_dict['URL'].append(url)                                                          # Loading the webpage by url
    time.sleep(2)
    
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
    
    try:
        brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      # Extracting Brand from xpath
        flip_dict["Brand"].append(brand.text.split()[0])
    except NoSuchElementException:
        flip_dict['Brand'].append('-')
        
    try:
        price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Brand from xpath
        flip_dict['Price'].append(price.text)
    except NoSuchElementException:
        flip_dict['Price'].append('-')
        
    try:
        name = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      # Extracting Brand from xpath
        flip_dict['Smartphone'].append(name.text)
    except NoSuchElementException:
        flip_dict['Smartphone'].append('-')
    
    try:
        color = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      # Extracting Name from xpath
        flip_dict['Colour'].append(color.text)
    except NoSuchElementException:
        flip_dict['Colour'].append('-')
    
    try:
        disp_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  # Extracting Ratings from xpath
        flip_dict['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        flip_dict['Display Size'].append('-')
    
   
    try:
        rom = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         # Extracting Expected Delivery from xpath
        flip_dict['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        flip_dict['Storage(ROM)'].append('-')
    
    try:
        ram = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['RAM'].append(ram.text)
    except NoSuchElementException:
        flip_dict['RAM'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        pri_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        flip_dict['Primary Camera'].append(pri_cam.text)
    except NoSuchElementException:
        flip_dict['Primary Camera'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        cam_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        flip_dict['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        flip_dict['Secondary Camera'].append('-')
        
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Capacity'].append(bat_cap.text)
    except NoSuchElementException:
        flip_dict['Battery Capacity'].append('-')
    
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[2]/ul/li')
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[1]')
                if bat_chk.text != "Battery Type" : raise NoSuchElementException
                bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr[2]/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
            if bat_chk.text != "Battery Type" : raise NoSuchElementException
            bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Type'].append(bat_typ.text)
    except NoSuchElementException:
        flip_dict['Battery Type'].append('-')

Scraping URL =  https://www.flipkart.com/google-pixel-4a-just-black-128-gb/p/itm023b9677aa45d?pid=MOBFUSBNAZGY7HQU&lid=LSTMOBFUSBNAZGY7HQUWHTF0C&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=SEARCH&iid=af9a248d-4e06-4039-a699-5e0934a7d1ef.MOBFUSBNAZGY7HQU.SEARCH&ppt=hp&ppn=homepage&ssid=4jgf6oqyg00000001640861963009&qH=9b26a23b2cff510d


C:\Users\hp\AppData\Local\Temp/ipykernel_1180/5174876.py:9: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/5174876.py:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  brand = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      # Extracting Brand from xpath
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/5174876.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  price = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Brand from xpath
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/5174876.py:27: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  name = driver.find_element_by_xpath('//

Scraping URL =  https://www.flipkart.com/redmi-10-prime-phantom-black-64-gb/p/itm9377b4c936baa?pid=MOBG6RGHHA2DGZZY&lid=LSTMOBG6RGHHA2DGZZYYZMW3D&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=SEARCH&iid=af9a248d-4e06-4039-a699-5e0934a7d1ef.MOBG6RGHHA2DGZZY.SEARCH&ppt=hp&ppn=homepage&ssid=4jgf6oqyg00000001640861963009&qH=9b26a23b2cff510d


C:\Users\hp\AppData\Local\Temp/ipykernel_1180/5174876.py:91: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/5174876.py:93: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/5174876.py:111: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr[2]/td[1]')
C:\Users\hp\AppData\Local\Temp/ipykernel_1180/5174876.py:113: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bat_typ = driver.find_el

Scraping URL =  https://www.flipkart.com/samsung-galaxy-m31-space-black-128-gb/p/itmed6043337ac0e?pid=MOBFUYQTCWZVMGEK&lid=LSTMOBFUYQTCWZVMGEKQRCZFI&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=search&fm=SEARCH&iid=af9a248d-4e06-4039-a699-5e0934a7d1ef.MOBFUYQTCWZVMGEK.SEARCH&ppt=hp&ppn=homepage&ssid=4jgf6oqyg00000001640861963009&qH=9b26a23b2cff510d


C:\Users\hp\AppData\Local\Temp/ipykernel_1180/5174876.py:103: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  bat_typ = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr[2]/td[2]/ul/li')


Scraping URL =  https://www.flipkart.com/tecno-spark-7t-nebula-orange-128-gb/p/itm5b5721a5b3932?pid=MOBG7F6RUGHFTDPS&lid=LSTMOBG7F6RUGHFTDPSFG3YDJ&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_4&otracker=search&otracker1=search&fm=SEARCH&iid=af9a248d-4e06-4039-a699-5e0934a7d1ef.MOBG7F6RUGHFTDPS.SEARCH&ppt=hp&ppn=homepage&ssid=4jgf6oqyg00000001640861963009&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/redmi-9-power-fiery-red-64-gb/p/itmcd78b9bc04aa9?pid=MOBFYZF8XVSHPK5M&lid=LSTMOBFYZF8XVSHPK5MZX9ZGZ&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_5&otracker=search&otracker1=search&fm=SEARCH&iid=af9a248d-4e06-4039-a699-5e0934a7d1ef.MOBFYZF8XVSHPK5M.SEARCH&ppt=hp&ppn=homepage&ssid=4jgf6oqyg00000001640861963009&qH=9b26a23b2cff510d
Scraping URL =  https://www.flipkart.com/tecno-spark-8-atlantic-blue-64-gb/p/itm42f4215492706?pid=MOBG74PTYVGHDZPG&lid=LSTMOBG74PTYVGHDZPG9PB2VW&marketplace=FLIPKART&q=pixel+4A&store=tyy%2F4io&srno=s_1_6&otracker=search&otracker

In [257]:
print(len(flip_dict["Brand"]), len(flip_dict["Smartphone"]), len(flip_dict["Price"]), len(flip_dict['URL']))

24 24 24 24


In [258]:
flip_df = pd.DataFrame.from_dict(flip_dict)
flip_df

,Brand,Smartphone,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Battery Type,Price,URL
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),3140 mAh,-,"₹28,999",https://www.flipkart.com/google-pixel-4a-just-...
1,REDMI,10 Prime,Phantom Black,4 GB,64 GB,50MP + 8MP + 2MP + 2MP,8MP Front Camera,16.51 cm (6.5 inch),6000 mAh,Li-Po,"₹13,103",https://www.flipkart.com/redmi-10-prime-phanto...
2,SAMSUNG,Galaxy M31,Space Black,8 GB,128 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),6000 mAh,lithium-ion,"₹20,948",https://www.flipkart.com/samsung-galaxy-m31-sp...
3,Tecno,Spark 7T,Nebula Orange,4 GB,128 GB,48MP Rear Camera,8MP Front Camera,16.56 cm (6.52 inch),6000 mAh,Li-Po,"₹10,943",https://www.flipkart.com/tecno-spark-7t-nebula...
4,REDMI,9 Power,Fiery Red,4 GB,64 GB,48MP + 8MP + 2MP + 2MP,8MP Front Camera,16.59 cm (6.53 inch),6000 mAh,Lithium-polymer,"₹13,305",https://www.flipkart.com/redmi-9-power-fiery-r...
5,Tecno,Spark 8,Atlantic Blue,2 GB,64 GB,16MP Rear Camera,8MP Front Camera,16.56 cm (6.52 inch),5000 mAh,Li-Po,"₹8,398",https://www.flipkart.com/tecno-spark-8-atlanti...
6,Redmi,Redmi 9,Sky Blue,4 GB,64 GB,13MP + 2MP,5MP Front Camera,16.59 cm (6.53 inch),5000 mAh,Li-Po,"₹10,790",https://www.flipkart.com/redmi-9-sky-blue-64-g...
7,Redmi,Redmi 9A,SeaBlue,2 GB,32 GB,-,-,16.59 cm (6.53 inch),5000 mAh,-,"₹7,776",https://www.flipkart.com/redmi-9a-seablue-32-g...
8,REDMI,REDMI Note 10 lite,Glacier White,4 GB,64 GB,Primary Camera,-,16.94 cm (6.67 inch),-,-,"₹14,445",https://www.flipkart.com/redmi-note-10-lite-gl...
9,LAVA,Z6,AQUA BLUE,6 GB,64 GB,Primary Camera,-,16.51 cm (6.5 inch),-,-,"₹9,990",https://www.flipkart.com/lava-z6-aqua-blue-64-...


Q5 Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.


In [260]:
# opening google maps
import warnings
warnings.filterwarnings('ignore')
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element_by_id("searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element_by_id("searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City Name : Delhi
URL Extracted:  https://www.google.co.in/maps/place/Delhi/@28.6629151,77.4751604,14z/data=!4m5!3m4!1s0x390cfd5b347eb62d:0x37205b715389640!8m2!3d28.7040592!4d77.1024902
Latitude = 28.6629151, Longitude = 77.4751604


Q6 Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in.

In [261]:
driver.get('https://trak.in/')
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

In [283]:
fund_dic = {}
fund_dic['Date'] = []
fund_dic['Startup Name'] = []
fund_dic['Industry/Vertical'] = []
fund_dic['Sub-Vertical'] = []
fund_dic['Location'] = []
fund_dic['Investor'] = []
fund_dic['Investment Type'] = []
fund_dic['Amount(in USD)'] = []

In [284]:
for i in range(3,12):
    driver.find_element_by_xpath('//div[@id="tablepress-54_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-54"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dic['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-54"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dic['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-54"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dic['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-54"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dic['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-54"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dic['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-54"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dic['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-54"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dic['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-54"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dic['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dic)
fund_df


,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
...,...,...,...,...,...,...,...,...
67,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
68,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
69,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
70,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"


In [287]:
fund_d = {}
fund_d['Date'] = []
fund_d['Startup Name'] = []
fund_d['Industry/Vertical'] = []
fund_d['Sub-Vertical'] = []
fund_d['Location'] = []
fund_d['Investor'] = []
fund_d['Investment Type'] = []
fund_d['Amount(in USD)'] = []

In [288]:
for i in range(3,12):
    driver.find_element_by_xpath('//div[@id="tablepress-55_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-55"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_d['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-55"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_d['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-55"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_d['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-55"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_d['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-55"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_d['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-55"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_d['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-55"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_d['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-55"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_d['Amount(in USD)'].append(a.text)
    
fund_df2 = pd.DataFrame(fund_d)
fund_df2

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
1,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"
2,19/02/2021,Fingerlix,Hospitality,Semi-cooked food delivery app,Mumbai,"Rhodium Trust, Accel Partners and Swiggy",Series C,"2,747,045.20"
3,17/02/2021,Zolve,FinTech,Global Neobank Venture,Mumbai,Accel Partners and Lightspeed Venture Partners,Seed,"1,50,00,000"
4,15/02/2021,KreditBee,Finance,Digital lending platform,Bengaluru,Azim Premji’s PremjiInvest and South Korea’s M...,Series C,"75,000,000"
...,...,...,...,...,...,...,...,...
76,15/02/2021,KreditBee,Finance,Digital lending platform,Bengaluru,Azim Premji’s PremjiInvest and South Korea’s M...,Series C,"75,000,000"
77,12/02/2021,Pepperfry,E-commerce,Multi-brand furniture brand,Mumbai,InnoVen Capital,Debt Financing,"4,773,958"
78,12/02/2021,Grofers,E-Commerce,Online supermarket,Gurgaon,SoftBank Vision Fund (SVF),Unspecified,"55,000,000"
79,09/02/2021,Nothing,Technology,Consumer Technology Venture,London,GV,Series A,"15,000,000"


In [289]:
fund_d1 = {}
fund_d1['Date'] = []
fund_d1['Startup Name'] = []
fund_d1['Industry/Vertical'] = []
fund_d1['Sub-Vertical'] = []
fund_d1['Location'] = []
fund_d1['Investor'] = []
fund_d1['Investment Type'] = []
fund_d1['Amount(in USD)'] = []

In [290]:
for i in range(3,12):
    driver.find_element_by_xpath('//div[@id="tablepress-56_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-56"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_d1['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-56"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_d1['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-56"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_d1['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-56"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_d1['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-56"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_d1['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-56"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_d1['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-56"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_d1['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-56"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_d1['Amount(in USD)'].append(a.text)
    
fund_df3 = pd.DataFrame(fund_d1)
fund_df3

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur, Rajasthan",Innoven Capital,Debt Financing,"250,000,000"
1,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),Palo Alto,Sorenson Capital Partners,Series D,"140,000,000"
2,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
3,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
4,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
...,...,...,...,...,...,...,...,...
58,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,Krishtal Advisors Pte Ltd,Series E,"8,000,000"
59,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MC Global Edtech, B Capital, Baron, others",Series F,"460,000,000"
60,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"Titan Capital, others",Seed,"300,000,000"
61,25/03/2021,CityMall,E-commerce,Social ecommerce and online grocery platform,Gurgaon,Accel Partners,Series A,"11,000,000"


In [291]:
frames = [fund_df,fund_df2,fund_df3 ]
df_keys = pd.concat(frames, keys=['january', 'faburary','march'])

df_keys

Date            Startup Name       Industry/Vertical  \
january 0   15/01/2021         Digit Insurance      Financial Services   
        1   28/01/2021  Bombay Shaving Company  Consumer Goods Company   
        2   19/01/2021                  DeHaat        AgriTech Startup   
        3   19/01/2021               Darwinbox                    SaaS   
        4   18/01/2021                   mfine     Health Tech Startup   
...                ...                     ...                     ...   
march   58  30/03/2021                   Dunzo              E-commerce   
        59  30/03/2021                  BYJU’S                Edu-tech   
        60  23/03/2021             SkilloVilla                Edu-tech   
        61  25/03/2021                CityMall              E-commerce   
        62  26/03/2021                   DotPe                 FinTech   

                                                 Sub-Vertical   Location  \
january 0                                  Insurance Services  Bengaluru   
        1       Shave care, beard care, and skincare products  New Delhi   
        2   online marketplace for farm products and services      Patna   
        3                                             HR Tech     Mumbai   
        4                  AI-powered telemedicine mobile app  Bengaluru   
...                                                       ...        ...   
march   58                           Hyper-local delivery app  Bengaluru   
        59                                    Online tutoring  Bengaluru   
        60                Career and job-oriented upskilling.  Bengaluru   
        61       Social ecommerce and online grocery platform    Gurgaon   
        62  Commerce and payments platform to offline ente...    Gurgaon   

                                                    Investor Investment Type  \
january 0   A91 Partners, Faering Capital, TVS Capital Funds         Venture   
        1                                  Reckitt Benckiser         Venture   
        2                                    Prosus Ventures        Series C   
        3                                Salesforce Ventures            Seed   
        4                         Heritas Capital Management   Venture Round   
...                                                      ...             ...   
march   58                         Krishtal Advisors Pte Ltd        Series E   
        59        MC Global Edtech, B Capital, Baron, others        Series F   
        60                             Titan Capital, others            Seed   
        61                                    Accel Partners        Series A   
        62                                              PayU        Series A   

           Amount(in USD)  
january 0     1,80,00,000  
        1    6,172,258.50  
        2      30,000,000  
        3      15,000,000  
        4      16,000,000  
...                   ...  
march   58      8,000,000  
        59    460,000,000  
        60    300,000,000  
        61     11,000,000  
        62     27,500,000  

[216 rows x 8 columns]

In [293]:
df_keys.to_csv("Indian Startups_Q1_2021.csv")

Q7 Write a program to scrap all the available details of best gaming laptops from digit.in

In [304]:
#Activating the chrome browser
driver = webdriver.Chrome(r"C:/Users/hp/Downloads/chromedriver.exe")

In [305]:
#Opening the specified url
url = "https://www.digit.in/"
driver.get(url)
time.sleep(3)
#searching for best laptop
best_gam_lap = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

In [306]:
#Creating empty lists
lap_name = []
ope_sys = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graph_proc = []
price = []

In [307]:
# Scraping the data of laptop names
name_tags = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
for name in name_tags:
    lap_name.append(name.text)
    
    
#Scraping the data of operating system
try:
    os_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
    for os in os_tags:
        ope_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display
try:
    disp_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
    for disp in disp_tags:
        display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    pro_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[5]//td[3]")
    for pro in pro_tags:
        processor.append(pro.text)
except NoSuchElementException:
    pass


#Scraping data of memory
try:
    memo_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[6]//td[3]")
    for memo in memo_tags:
        memory.append(memo.text)
except NoSuchElementException:
    pass


#Scraping data of weight
try:
    wgt_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[7]//td[3]")
    for wgt in wgt_tags:
        weight.append(wgt.text)
except NoSuchElementException:
    pass


#Scraping data of dimensions
try:
    dim_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[8]//td[3]")
    for dim in dim_tags:
        dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    gra_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[9]//td[3]")
    for gra in gra_tags:
        graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    pri_tags = driver.find_elements_by_xpath("//td[@class='smprice']")
    for pri in pri_tags:
        price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [308]:
#DATA FRAMEING
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = lap_name
Gaming_Laptop['Operating system'] = ope_sys
Gaming_Laptop['Display'] = display
Gaming_Laptop['Processor'] = processor
Gaming_Laptop['Memory'] = memory
Gaming_Laptop['Weight'] = weight
Gaming_Laptop['Dimensions'] = dimensions
Gaming_Laptop['Graphical Processor'] = graph_proc
Gaming_Laptop['Price'] = price

In [309]:
#Printing data frame
Gaming_Laptop

,Laptop Name,Operating system,Display,Processor,Memory,Weight,Dimensions,Graphical Processor,Price
0,Acer NITRO 5 Ryzen 9 (2021),Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,"Rs 129,990"
1,MSI Stealth 15M 11th Gen Core i7-11375H (2021,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,"Rs 134,990"
2,ASUS ROG Strix Scar 15 Ryzen 9-5900HX (2021),Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,"Rs 268,990"
3,Alienware Area 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Gen Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,"Rs 342,989"
4,Alienware m15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Gen Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,"Rs 319,990"
5,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,"Rs 215,990"
6,Asus ROG Zephyrus G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"Rs 164,990"
7,Lenovo Legion 5i,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Gen Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,"Rs 76,988"
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"Rs 185,000"
9,Acer Aspire 7 gaming,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"Rs 57,600"


In [310]:
Gaming_Laptop.to_csv("Gaming_Laptop.csv")

Q8 Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.


In [319]:
#Activating the chrome browser
driver = webdriver.Chrome(r"C:/Users/hp/Downloads/chromedriver.exe")
time.sleep(2)

In [320]:
#get the specified url
url = "https://www.forbes.com/?sh=69e6b8c92254"
driver.get(url)
time.sleep(3)

#clicking the explore button
button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
button.click()
time.sleep(1)

#select billionaire  
bill = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
bill.click()
time.sleep(1)

#select world billionaire  
world_bill= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_bill.click()
time.sleep(1)


In [321]:
#Scraping required data from the web page
#Creating empty lists
Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []


while(True):
    #Scraping the data of rank
    rank_tags= driver.find_elements_by_xpath("//div[@class='rank']")
    for rank in rank_tags:
        Rank.append(rank.text)
    time.sleep(1)
    
    
    #Scraping the data of names
    name_tags= driver.find_elements_by_xpath("//div[@class='personName']//div")
    for name in name_tags:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    #Scraping data of age
    age_tags= driver.find_elements_by_xpath("//div[@class='age']//div")
    for age in age_tags:
        Age.append(age.text)   
    time.sleep(1)
    
    
    #Scraping data of citizenship
    cit_tags= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for cit in cit_tags:
        citizenship.append(cit.text)
    time.sleep(1)
    
    
    #Scraping data of source of income
    sour_tags= driver.find_elements_by_xpath("//div[@class='source']")
    for sour in sour_tags:
        Source.append(sour.text)    
    time.sleep(1)
    
    
    #Scraping data of Industry
    ind_tags= driver.find_elements_by_xpath("//div[@class='category']//div")
    for ind in ind_tags:
        industry.append(ind.text)
        
        
    #scraping data of net_worth of billionaire 
    net_tags= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for net in net_tags:
        total_net_worth.append(net.text)
    time.sleep(1)
    
    
    #Clicking on next button
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break
        
#Scraping data of net worth
Net_Worth = []
for i in range(0,len(total_net_worth),2):
    Net_Worth.append(total_net_worth[i])

In [322]:
#Framing data
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = Net_Worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry
#Printing dataframe
Billionaires


,Rank,Names,Net Worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


In [324]:
#Saving dataset to csv
Billionaires.to_csv("Billionaries.csv")

Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video

In [478]:
#Activating the chrome browser
driver = webdriver.Chrome(r"C:/Users/hp/Downloads/chromedriver.exe")
time.sleep(2)

In [479]:
# Opening the youtube.com
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

In [480]:
#finding element for search bar
search_video=driver.find_element_by_xpath("//input[@id='search']")

search_video.send_keys("PSY - GANGNAM STYLE(강남스타일) M/V")  #entering Video name
time.sleep(1)

In [481]:
search_bar=driver.find_element_by_xpath("//button[@id='search-icon-legacy']")

In [482]:
search_bar.click()

In [483]:
title_video=driver.find_element_by_xpath("//a[@id='video-title']//yt-formatted-string")
title_video.click()

In [503]:
driver.execute_script("window.scrollTo(0,100000000000)")
time.sleep(10)

In [518]:
cmnt=driver.find_elements_by_xpath("//yt-formatted-string[@id='content-text']")
cmnt

[<selenium.webdriver.remote.webelement.WebElement (session="496e930c24d662ba2ddf30e4d47390c1", element="58b8edb3-14cc-4776-9493-9a7fd889140c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496e930c24d662ba2ddf30e4d47390c1", element="c9b0b8c4-bd2d-4db5-9cf1-37a0c0058253")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496e930c24d662ba2ddf30e4d47390c1", element="b862ae5b-d727-4e66-9b05-e77a5c059b62")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496e930c24d662ba2ddf30e4d47390c1", element="d8a7f0f9-2fff-43f5-a16b-001869575fa7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496e930c24d662ba2ddf30e4d47390c1", element="58575bef-c938-475a-a706-b3ce0463523c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496e930c24d662ba2ddf30e4d47390c1", element="bc8fd04e-6958-499d-b431-38c03e6bc17c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="496e930c24d662ba2ddf30e4d47390c1", element="2085745b-84cd-4065-82aa-c6

In [519]:
cmnts=[]
for i in cmnt:
    cmnts.append(i.text)
cmnts   

['For those who dont know: This song literally broke the internet.',
 "we didn't realise how good we had it back in the 2010-2013 era..",
 'The dance moves makes it more awesome',
 'Saudades dessa época! Aeee cadê os BR??? 🇧🇷🇧🇷🇧🇷',
 'Old is Gold! 🏅',
 'This song never gets old. No matter how much I listen, I never get bored.',
 'Me in 2012: ha funny song\n\nMe in 2019: cries in nostalgia',
 'Как же в 2012 было круто!',
 '9 years later and this song still hits different. Truly a masterpiece.',
 "Didn't realize we have been listening to kpop since we were like 7 years old",
 "This song is stupidity addictive I can't stop replaying it 🔥🔥",
 'Que nostalgia escucharla de nuevo. muy buena 😁',
 'After 9 years. This song is   immortal',
 'Popular opinion : This song never getting old & forever will be iconic.',
 'Essa música nunca perde a magia',
 'This song never gets old. No matter how much I listen, I never get bored.',
 'The only one K-pop song that the humanity actually accepts on this wo

In [520]:
len(cmnts)

554

In [521]:
upvote=driver.find_elements_by_xpath("//span[@id='vote-count-middle']")

In [522]:
upvotes=[]
for i in upvote:
    upvotes.append(i.text)
upvotes 

['12K',
 '2.4K',
 '3.9K',
 '12K',
 '7.7K',
 '88',
 '23K',
 '793',
 '2K',
 '170K',
 '2.4K',
 '131',
 '2.3K',
 '2.5K',
 '138',
 '26',
 '4.2K',
 '156K',
 '824',
 '2.2K',
 '1.6K',
 '710',
 '9',
 '25',
 '22',
 '709',
 '658',
 '729',
 '12',
 '12',
 '380',
 '27K',
 '33K',
 '7',
 '76',
 '4',
 '772',
 '5',
 '2.6K',
 '398',
 '595',
 '255',
 '197',
 '5',
 '13',
 '1',
 '498',
 '54',
 '720',
 '24',
 '80',
 '3',
 '46K',
 '258',
 '7',
 '2',
 '6',
 '2',
 '1.8K',
 '62',
 '71K',
 '283',
 '77',
 '6',
 '143',
 '3',
 '83K',
 '71',
 '43',
 '109',
 '439',
 '211',
 '763',
 '1',
 '29',
 '2',
 '53',
 '47',
 '4.6K',
 '49',
 '340',
 '88K',
 '3.1K',
 '47',
 '12',
 '37',
 '260',
 '19',
 '11',
 '122',
 '22',
 '21',
 '3.7K',
 '32',
 '2',
 '3',
 '16',
 '83',
 '2.6K',
 '97',
 '5.8K',
 '5',
 '3',
 '5',
 '4',
 '1',
 '246',
 '6',
 '4',
 '40',
 '32',
 '4',
 '1.9K',
 '11',
 '',
 '3',
 '3',
 '8',
 '247',
 '3',
 '213',
 '17',
 '3',
 '4',
 '37',
 '6',
 '171',
 '32',
 '40',
 '24',
 '63',
 '3',
 '326',
 '',
 '5',
 '1',
 '2',
 '4

In [523]:
len(upvotes)

560

In [524]:
Time_ago_posted=driver.find_elements_by_xpath("//yt-formatted-string[@class='published-time-text above-comment style-scope ytd-comment-renderer']")


In [525]:
time=[]
for i in Time_ago_posted:
    time.append(i.text)
time

['1 year ago',
 '4 days ago',
 '2 weeks ago',
 '2 years ago',
 '2 months ago',
 '22 hours ago',
 '2 years ago (edited)',
 '12 days ago',
 '2 weeks ago',
 '2 years ago (edited)',
 '3 weeks ago',
 '2 days ago',
 '5 months ago',
 '1 month ago',
 '3 days ago',
 '1 day ago',
 '2 months ago',
 '2 years ago (edited)',
 '2 years ago',
 '1 month ago',
 '2 years ago',
 '2 weeks ago',
 '2 hours ago',
 '2 days ago',
 '1 day ago',
 '3 weeks ago (edited)',
 '3 weeks ago',
 '1 month ago',
 '2 months ago',
 '3 days ago',
 '2 weeks ago',
 '2 years ago',
 '2 years ago (edited)',
 '1 day ago',
 '10 days ago',
 '2 days ago',
 '1 month ago',
 '6 hours ago',
 '11 months ago',
 '3 weeks ago',
 '2 years ago',
 '2 months ago',
 '12 days ago',
 '11 days ago (edited)',
 '2 days ago',
 '21 hours ago',
 '1 year ago',
 '7 days ago (edited)',
 '2 months ago (edited)',
 '5 days ago',
 '3 weeks ago',
 '2 days ago',
 '2 years ago (edited)',
 '2 weeks ago (edited)',
 '2 days ago',
 '21 hours ago',
 '9 days ago',
 '1 day

In [526]:
len(time)

560

In [527]:
Nos=[]
for i in range(1,len(time)+1):
    Nos.append(i)

In [528]:
import pandas as pd
Youtube=pd.DataFrame({})
Youtube['Nos']=Nos[:500]
Youtube['500 COMMENTS']=cmnts[:500]
Youtube['UPVOTES']=upvotes[:500]
Youtube['TIME AGO POSTED']=time[:500]
Youtube.style.hide_index()

Nos,500 COMMENTS,UPVOTES,TIME AGO POSTED
1,For those who dont know: This song literally broke the internet.,12K,1 year ago
2,we didn't realise how good we had it back in the 2010-2013 era..,2.4K,4 days ago
3,The dance moves makes it more awesome,3.9K,2 weeks ago
4,Saudades dessa época! Aeee cadê os BR??? 🇧🇷🇧🇷🇧🇷,12K,2 years ago
5,Old is Gold! 🏅,7.7K,2 months ago
6,"This song never gets old. No matter how much I listen, I never get bored.",88,22 hours ago
7,Me in 2012: ha funny song Me in 2019: cries in nostalgia,23K,2 years ago (edited)
8,Как же в 2012 было круто!,793,12 days ago
9,9 years later and this song still hits different. Truly a masterpiece.,2K,2 weeks ago
10,Didn't realize we have been listening to kpop since we were like 7 years old,170K,2 years ago (edited)


In [544]:
#Saving dataset to csv
Youtube.to_csv("Youtube.csv")

Q10 Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description

In [530]:
! pip install selenium
import time
import warnings
warnings.filterwarnings('ignore')

In [531]:
#Activating the chrome browser
driver = webdriver.Chrome(r"C:/Users/hp/Downloads/chromedriver.exe")
time.sleep(2)

In [532]:
url="https://www.hostelworld.com/"
driver.get(url)

In [533]:
from selenium.common.exceptions import  StaleElementReferenceException,ElementClickInterceptedException
from selenium.common.exceptions import  NoSuchElementException,NoSuchWindowException

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [559]:
#locating the location search bar
search_bar = driver.find_element_by_id('search-input-field')

# Entering Lonodn in search bar
search_bar.send_keys("London")
time.sleep(3)

#Select london
London =driver.find_element_by_xpath("//div[@class='label']")
# Clicking on the button
London.click()
time.sleep(3)


# Do click on search button
search_btn = driver.find_element_by_id('search-button')
search_btn.click()

In [560]:
#lets find required data
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description =[]
product_url = []

In [561]:
#Scraping the required informations
for i in driver.find_elements_by_xpath("//div[@class = 'pagination-item pagination-current' or @class='pagination-item']"):
    i.click()
    time.sleep(4)
    
    #Scraping hostel name
    try:
        name = driver.find_elements_by_xpath("//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
        
    #Scraping distance from city centre
    try:
        dist = driver.find_elements_by_xpath("//div[@class='subtitle body-3']//a//span[1]")
        for i in dist:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
        
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    #Scraping privates from price
        try:
            pvt_price = driver.find_element_by_xpath("//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
            
    #Scraping dorms from price
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
        try:
            dorms = driver.find_element_by_xpath("//a[@class='prices']//div[2]//div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
            
    #Scraping facilities
    try:
        fac1 = driver.find_elements_by_xpath("//div[@class='has-wifi']")
        fac2 = driver.find_elements_by_xpath("//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
        facilities.append('-')
        
    #Fetching url of each hostel
    p_url = driver.find_elements_by_xpath("//div[@class='prices-col']//a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

for i in product_url:
    driver.get(i)
    time.sleep(3)
    #Do click on show more button for description
    try:
        driver.find_element_by_xpath("//a[@class='toggle-content']").click()
        time.sleep(5)
    except NoSuchElementException:
        pass
    
    #Scraping ratings
    try:
        rat = driver.find_element_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
        
    #Scraping total reviews   
    try:
        rws = driver.find_element_by_xpath("//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        
    #Fetching overall review
    try:
        overall_rw = driver.find_element_by_xpath("//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
        
    #Fetching property description 
    try:
        disc = driver.find_element_by_xpath("//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')

In [572]:
len(description)

79

In [573]:
#Creating dataframe
Hostels = pd.DataFrame({})
Hostels['Hostel_Name'] = hostel_name
Hostels['Distance fron city centre'] = distance
Hostels['Ratings'] = rating
Hostels['Total_reviews'] = reviews
Hostels['Overall Reviews'] = over_all
Hostels['Privates from price'] = pvt_prices
Hostels['Dorms from price'] = dorms_price
Hostels['Facilities'] = facilities[:79]
Hostels['Description'] = description
Hostels


,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Overall Reviews,Privates from price,Dorms from price,Facilities,Description
0,Mornington Camden,4.1km from city centre,8.3,44,Fabulous,Rs4358,Rs2593,"Free WiFi, Follows Covid-19 sanitation guidance","2 Hurdwick Place, London, England"
1,Smart Russell Square Hostel,2.6km from city centre,6.7,9561,Good,Rs4358,Rs2593,"Free WiFi, Follows Covid-19 sanitation guidance","71 Guilford Street, London, England"
2,Smart Camden Inn Hostel,4.4km from city centre,8.4,2731,Fabulous,Rs4358,Rs2593,"Free WiFi, Follows Covid-19 sanitation guidance","55/57 Bayham Street, Camden, London, England"
3,Queen Elizabeth Chelsea,5.7km from city centre,7.5,3212,Very Good,Rs4358,Rs2593,"Free WiFi, Follows Covid-19 sanitation guidance","58 Bagleys Lane, London, England"
4,Pickwick Hall,2.3km from city centre,9.0,2273,Superb,Rs4358,Rs2593,"Free WiFi, Follows Covid-19 sanitation guidance","7 Bedford Place, Bloomsbury, London, England"
...,...,...,...,...,...,...,...,...,...
74,Kip Hotel,Hotel - 6.8km from city centre,9.6,52,Superb,Rs3942,Rs2475,"Free WiFi, Follows Covid-19 sanitation guidance","2 Aspland Grove, London, England"
75,Green Rooms,Hotel - 10.8km from city centre,9.7,76,Superb,Rs3942,Rs2475,"Free WiFi, Follows Covid-19 sanitation guidance","13-27 Station Road, Wood Green, London, England"
76,23 Matheson Road,Bed and Breakfast - 6.2km from city centre,-,0,No Rating,Rs3942,Rs2475,"Free WiFi, Follows Covid-19 sanitation guidance","23 Matheson Road, London, England"
77,Park Hotel Essex,Hotel - 24.1km from city centre,8.0,109,Fabulous,Rs3942,Rs2475,"Free WiFi, Follows Covid-19 sanitation guidance","327 Cranbrook Road, Ilford, London, England"


In [574]:
#Saving dataset to csv file
Hostels.to_csv("London_Hostels.csv")